### Dropped from na02approximation.ipynb

Move to M104C


![Chebychev polynomial 3](https://jhparkyb.github.io/resources/notes/na/104ASlides_Approximation021.png)

##### Divided differences

![Divided differences 1](https://jhparkyb.github.io/resources/notes/na/104ASlides_Approximation026.png)

![Divided differences 2](https://jhparkyb.github.io/resources/notes/na/104ASlides_Approximation027.png)

![Divided differences 3](https://jhparkyb.github.io/resources/notes/na/104ASlides_Approximation028.png)

##### Horner's algorithm: efficient evaluation of interpolation

Efficient way of evaluating polynomails.

![Example of Horner's algorithm](https://jhparkyb.github.io/resources/notes/na/104ASlides_Approximation010.png)

![Finding interpolating polynomial in the Newton form 2](https://jhparkyb.github.io/resources/notes/na/104ABoardWork_Approximation002.png)

swapping
<!-- **Motivating example**

See the example below. There are no errors or bugs in the function `gauss_elimination2D`. But it produces something weird: `x_lib` is the correct solution, which is obtained by `numpy.linalg` (which uses LAPACK library). What has happened? -->

swapping
<!-- import numpy as np

def gauss_elimination2D(AA, bb):
    """Solves Ax = b using Gauss elimination.

    Args:
        A: A square matrix.
        b: A vector.

    Returns:
        x: A vector.
    """
    n = 2
    AA = A.copy()
    bb = b.copy()
    x = np.zeros(n).reshape(-1, 1)

    m = AA[1, 0] / AA[0, 0]
    AA[1, 1] -= m * AA[1, 0]
    bb[1] -= m * bb[0]
    
    x[1] = bb[1] / AA[1, 1]
    x[0] = (bb[0] - AA[0, 1] * x[1]) / AA[0, 0]
    
    return x

A = np.array([[10e-20,  1.], 
              [     1.,  2.]])
b = np.array([1., 4.]).reshape(-1, 1)

x_naive = gauss_elimination2D(A, b)
x_lib = np.linalg.solve(A, b)

print("x_naive: \n", x_naive)
print("x: \n", x) -->

<!-- #### Inner product

Inner product is a far-reaching concept that helps us tackle differential geometry, Fourier analysis, solutions of partial differential equations just to name a few. Here, we give just a bit more details to see if the $$(f,g)_w:=\int_a^b f(x)g(x)w(x)dx$$ is indeed an inner product. -->


#### Pivoting

<!-- **Motivating example**

Recall the homework problem, where we solved the following linear system by programming 2D version of Gaussian elimination.

$$
\left\{
\begin{aligned}
10^{-20} x_1+x_2 & =1 \\
x_1+2 x_2 & =4
\end{aligned}
\right. 
$$

If we programmed "right," it should produce $(x_1, x_2)\approx(0,1)$, while the package `numpy.linalg.solve` gave $(x_1, x_2)\approx(2,1)$. 

What is going on? -->

<!-- **Remark**

- The source of error is that the first coefficient `A[0,0]` is so small that the computer (using IEEE double precision) "thinks" it is 0. 

$$
\left\{
\begin{aligned}
10^{-20} x_1+x_2 & =1 \\
x_1+2 x_2 & =4
\end{aligned}
\right. 
\Rightarrow
\left\{
\begin{aligned}
0 x_1+x_2 & =1 \\
x_1+2 x_2 & =4
\end{aligned}
\right. 
$$

Then, the first equation leads to $x_2=1$, which in turn, leads to $x_1 = 0$. Hence, we have $(x_1, x_2)\approx(0,1)$. 

- However, if we carry the small coefficient $10^{-20}$ and meticulously solve the equation, we obtain $(x_1, x_2)\approx(2,1)$. -->

<!-- **Idea** (Swapping or partial pivoting)

- Push down a row whose pivot is small, and pull up the one with the largest coefficient in that column.
- This procedure is called *partial pivoting*. -->

**Remark**

According to Sauer (2017) Numerical Analysis 3rd ed., the practical difference among the different types of PDE is:
- parabolic and hyperbolic
  - PDE is defined on an open region.
  - Boundary conditions for one variable are specified at one end of the region (initial state) and the system is solved by moving away from the boundary. (Think of time evolution.)
- elliptic
  - PDE is defined on the entire boundary of a closed region.
  - Boundary conditions are imposed literally on the boundary of the region.

___

To do: INCLUDE MORE COMPLETE EXPOSITION

**Remark** (Why try to avoid `for` loop in Python)

- Python `int` is not purely an integer, it is an object. (See code below.)
- When `for` loop is implemented, it takes out only pure integer part and work. (More process)
- Python lacks JIT (just-in-time compilation). 
  - Python is an interpreter. Hence, it executes each line.
  - But for `for` loop, interpreting each line is waste of time. We want just repeat the same task, and carry out the whole `for` loop as a single command.
  - If those lines are skipped while interpreting, and executed directly during the run time, a lot of computing time can be saved.

In [ ]:
# Python int is not purely an integer, it is an object.

i = 34

print(type(i))
print(id(i))
print(i.to_bytes(length=1))

<class 'int'>
9822216
b'"'


___

**Example** (Rewriting to avoid loss of significand)

Suppose that your project involves the following computation near $x = 0$ for a fixed value $a > 0$: 

$$
\sqrt{x^2 + a^2} - a .
$$

Rewrite this expression to avoid loss of significant digits along the computation.


**Answer**

The quantity can be rewritten as 

$$
\left(\sqrt{x^2+a^2}-a\right)\left(\frac{\sqrt{x^2+a^2}+a}{\sqrt{x^2+a^2}+a}\right)=\frac{x^2}{\sqrt{x^2+a^2}+a}
$$

We can use the last term in place of what's given in the example.

Let us assume the following, which is totally made up for illustrative purposes. 

- Our computer uses decimals, instead of binary.
- Our computer uses six-decimal-digit mantissa, including the integer part, floating point number system along with power of tens as scalar.
- $x=0.01$ and $a = 3$

Then, the input will be stored as

- $x^2 + a^2 = 9.00010 \times 10^0$
- $a = 3.00000 \times 10^0$

1. Original formula

Even assuming infinite precision for evaluating square root, the square root is stored as

- $\sqrt{x^2 + a^2} = 3.00002 \times 10^0$ (more precise value is $\sqrt{9.00010}=3.0000166666203705$)
- $\sqrt{x^2 + a^2} - a = 0.00002 = 2.0\times 10^{-5}$. $\longrightarrow$ (Only one significant digit) 

2. Rewritten formula

$$
\frac{x^2}{\sqrt{x^2+a^2}+a} = \frac{0.0001}{3.00002 + 3.00000} = 4.10000 \times 10^{-5}
$$

(More precise value is $\frac{0.0001}{3.00002 + 3.00000} = 0.00004082476100517466$)

In [ ]:
import numpy as np
x = 9.0001
a = 3.
print(np.sqrt(x))
print(0.0001/(np.sqrt(3.00002 + 3.00000)))

3.0000166666203705
4.082476100517466e-05


___

#### Python package for least square method (To be desired)

In [ ]:
import numpy as np

# 1. Create A and b for a least square problem
A = np.arange(3*5, dtype=np.float64).reshape((3,5))**2
b = np.arange(5, dtype=np.float64).reshape((5,1))**2

# 2. Solve the least square problem using pseudo inverse
x = np.linalg.pinv(A.T)@b

# 3. Check the result
print(A.T)
print(b)
print(x)

# 4. Check the residual
res = A.T@x - b
print(res)

# 5. Check the residual for the true solution (cheating)
x_ = np.array([1,0,0]).reshape((3,1))
print(A.T@x_ - b)

[[  0.  25. 100.]
 [  1.  36. 121.]
 [  4.  49. 144.]
 [  9.  64. 169.]
 [ 16.  81. 196.]]
[[ 0.]
 [ 1.]
 [ 4.]
 [ 9.]
 [16.]]
[[1.]
 [0.]
 [0.]]
[[0.00000000e+00]
 [1.33226763e-15]
 [5.32907052e-15]
 [1.24344979e-14]
 [2.13162821e-14]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]


___

**Theorem** (Projection matrix)

For a matrix $A\in\mathbb{R}^{m\times n}$, 

$$
P=A\left(A^T A\right)^{-1} A^T
$$ 

defines the orthogonal projection from $\mathbb{R}^{m}$ onto column space of $A$. In particular, if $A=v$ (a single column), then 

$$
P=\frac{v v^T}{v^T v}
$$

defines the orthogonal projection from ${R}^{m}$ onto the line $\{\lambda v \ : \  \lambda \in \mathbb{R} \}$.

<!-- **Proof**

From the least square discussion, we already know that the vector $\bar{x} =\left(A^T A\right)^{-1} A^T b$ yields the projection of $b$ onto the column space of $A$. Thus, taking the linear combination of columns of $A$ with $\bar x$ being coefficients, we obtain the projected vector of $b$. In other words, the map given by 

$$
b \mapsto A\left(A^T A\right)^{-1} A^T b (=A\bar{x})
$$

defines a projection onto column space of $A$.

In particular, if $A$ consists of a single column $v$, then this reads 

$$
b \mapsto \frac{v v^T}{v^T v} b
$$

 -->


___

<!-- Numerical issue with classical Gram-Schmidt

```
d = 1e-10
A = np.array([[1, d, 0, 0], [1, 0, d, 0], [1, 0, 0, d]], dtype=np.float64)

A = A.T

Q, R = qr_red_GS(A)

print("Q^TQ\n", Q.T @ Q) # gives non-orthogonal for classical Gram-Schmidt
``` -->